In [1]:
import pickle
import nltk

# Load the 8085 n-gram model
with open('n_gram_8085.pkl', 'rb') as f:
    model = pickle.load(f)

n = 3  # n-gram size

def predict_next_word(context):
    context = context[-n+1:]
    predictions = model.context_counts(model.vocab.lookup(context))
    return sorted(predictions.items(), key=lambda x: -x[1])[:5]

# Example usage
print("8085 Assembly Next Word Predictions:")
print(predict_next_word(['mov', 'a', ',']))

8085 Assembly Next Word Predictions:
[('01h', 1)]


In [2]:
import pickle
import nltk

# Load the 8085 n-gram model
with open('n_gram_8085.pkl', 'rb') as f:
    model = pickle.load(f)

n = 3  # n-gram size

def predict_next_word(context):
    context = context[-n+1:]
    predictions = model.context_counts(model.vocab.lookup(context))
    return sorted(predictions.items(), key=lambda x: -x[1])[:5]

# Example usage
print("8085 Assembly Next Word Predictions:")
print(predict_next_word(['mov', 'a', ',']))

8085 Assembly Next Word Predictions:
[('01h', 1)]


In [3]:
import pickle
import nltk

# Load the 8051 n-gram model
with open('n_gram_8051.pkl', 'rb') as f:
    model = pickle.load(f)

n = 3  # n-gram size

def predict_next_word(context):
    context = context[-n+1:]
    predictions = model.context_counts(model.vocab.lookup(context))
    return sorted(predictions.items(), key=lambda x: -x[1])[:5]

# Example usage
print("8051 Assembly Next Word Predictions:")
print(predict_next_word(['mov', 'a', ',']))

8051 Assembly Next Word Predictions:
[('b', 12), ('00h', 8), ('19h', 8), ('13h', 8), ('00000001b', 8)]


In [ ]:
import sys
import pickle
import json
import os
import traceback

def predict_next_word(model, context, n=3):
    """Generalized prediction function that works with any model"""
    context = context[-n+1:] if len(context) > 0 else []
    
    if len(context) > 0:
        predictions = model.context_counts(model.vocab.lookup(context))
        return sorted(predictions.items(), key=lambda x: -x[1])[:5]
    else:
        # Handle empty context case
        return [("nop", 1)]

try:
    # Get input from command line
    if len(sys.argv) < 2:
        print(json.dumps({"success": False, "error": "No input provided"}))
        sys.exit(1)
        
    # Get the raw input
    input_arg = sys.argv[1]
    
    # Check if input is a file path (starts with @)
    if input_arg.startswith('@'):
        file_path = input_arg[1:]  # Remove the @ symbol
        print(f"Reading input from file: {file_path}", file=sys.stderr)
        
        try:
            with open(file_path, 'r') as f:
                input_arg = f.read()
            print(f"Input from file: {repr(input_arg)}", file=sys.stderr)
        except Exception as e:
            print(f"Error reading file: {e}", file=sys.stderr)
            print(json.dumps({"success": False, "error": f"Error reading input file: {str(e)}"}))
            sys.exit(1)
    
    # Debug the raw input
    print(f"Raw input: {input_arg}", file=sys.stderr)
    
    # Pre-process input to handle quote issues
    # Windows command line often has issues with quotes
    input_json = input_arg
    
    # Debug the processed input
    print(f"Processed input: {input_json}", file=sys.stderr)
    
    try:
        data = json.loads(input_json)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}", file=sys.stderr)
        print(f"Input was: {input_json}", file=sys.stderr)
        print(json.dumps({"success": False, "error": f"JSON decode error: {str(e)}"}))
        sys.exit(1)
    
    # Get model type and context from input
    model_type = data.get('model', '8051')  # Default to 8051 if not specified
    context = data.get('context', [])
    
    # Validate model type
    supported_models = ['8051', '8085']
    if model_type not in supported_models:
        print(f"Unsupported model type: {model_type}. Using default 8051.", file=sys.stderr)
        model_type = '8051'
    
    print(f"Parsed data: model={model_type}, context={context}", file=sys.stderr)
    
    # Determine the absolute path to the model files
    script_dir = os.path.dirname(os.path.abspath(__file__))
    model_path = os.path.join(script_dir, f"../assets/pickles/n_gram_{model_type}.pkl")
    
    print(f"Looking for model at: {model_path}", file=sys.stderr)
    
    # Check if model file exists
    if not os.path.exists(model_path):
        print(f"Model file not found at: {model_path}, using mock data", file=sys.stderr)
        # Provide mock predictions based on the model type
        if context and context[0] == "mov":
            if model_type == '8051':
                mock_predictions = [("a", 10), ("b", 8), ("r1", 6), ("dptr", 4), ("c", 2)]
            else:  # 8085
                mock_predictions = [("a", 10), ("b", 8), ("h", 6), ("l", 4), ("m", 2)]
        else:
            mock_predictions = [("mov", 12), ("add", 8), ("jmp", 6), ("inc", 4), ("ret", 2)]
        
        print(json.dumps({"success": True, "predictions": mock_predictions, "model": model_type}))
        sys.exit(0)
    
    # If we reach here, try to use the actual model
    try:
        # Load the appropriate model
        with open(model_path, 'rb') as f:
            model = pickle.load(f)
        
        # Make prediction using our generalized function
        result = predict_next_word(model, context)
        
        # Return the result as JSON
        print(json.dumps({"success": True, "predictions": result, "model": model_type}))
        
    except Exception as e:
        print(f"Error using model: {e}", file=sys.stderr)
        traceback.print_exc(file=sys.stderr)
        
        # Fall back to mock data based on model type
        if context and context[0] == "mov":
            if model_type == '8051':
                mock_predictions = [("a", 10), ("b", 8), ("r1", 6), ("dptr", 4), ("c", 2)]
            else:  # 8085
                mock_predictions = [("a", 10), ("b", 8), ("h", 6), ("l", 4), ("m", 2)]
        else:
            mock_predictions = [("mov", 12), ("add", 8), ("jmp", 6), ("inc", 4), ("ret", 2)]
        
        print(json.dumps({"success": True, "predictions": mock_predictions, "model": model_type}))
        
except Exception as e:
    # Catch all other exceptions
    print(f"Unexpected error: {str(e)}", file=sys.stderr)
    traceback.print_exc(file=sys.stderr)
    print(json.dumps({"success": False, "error": str(e)}))
    sys.exit(1)